dictionary_const :
  - key : 한글단어(str)
  - value : list(str) -> [용어영문명, 영문약어명, 정의]
  
return_format

In [14]:
# definition_dictionary crawling
def get_definition(text) :
    from selenium import webdriver
    import urllib.request
    import requests
    import time
    
    chromedriver_dir = 'C:/dev/driver/chromedriver.exe'
    driver = webdriver.Chrome(chromedriver_dir)
    
    driver.get("https://ko.dict.naver.com/#/main")
    
    driver.find_element_by_id('ac_input').send_keys(text)
    
    driver.find_element_by_xpath("//button[@class='btn_search']").click()
    
    time.sleep(1)
    driver.find_element_by_xpath("//div[@class='component_keyword has-saving-function']/div[@class='row']/div[@class='origin']/a").click()
    time.sleep(1)
    
    try :
        output_ = driver.find_element_by_xpath("//ul[@class='mean_list my_mean_list']").text
    except : 
        output_ = driver.find_element_by_xpath("//div[@class='mean_desc']/p[@class='cont']").text
    finally :
        driver.close()
    
    return output_

def translate_by_papago_api(text) :
    import os
    import sys
    import urllib.request
    import json
    
    client_id = "bONH_8tRvIGnJbn7o966"
    client_secret = "zuoA87y5rv"
    
    output_ = ''
    encText = urllib.parse.quote(text)
    data = "source=ko&target=en&text=" + encText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        str2dict = json.loads(response_body.decode('utf-8'))
        output_ = str2dict['message']['result']['translatedText']
    else:
        output_ = "Error Code:" +rescode

    return output_
    

In [1]:
def getDefDict(path = 'dataset/행정표준용어.xls') :
    # convert dataframe_to_dictionary
    # key = 용어명, value = [용어영문명, 영문약어명, 정의]
    import pandas as pd
    from collections import defaultdict
    tmp = pd.read_excel(path, header=1, index_col=0)
    
    newDict = defaultdict(list)
    for idx in tmp.index :
        newDict[tmp.at[idx,'용어명']] = [tmp.at[idx,'용어영문명'],tmp.at[idx,'영문약어명'],tmp.at[idx,'정의'] ]
        
    return newDict

In [2]:
dict_ = getDefDict('C:/dev/행정표준용어.xls')

C:\dev\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\dev\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\dev\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
def getWordDict(path) :
    import pandas as pd
    from collections import defaultdict
    tmp = pd.read_excel(path, sheet_name='단어')
    
    tmpDict = defaultdict(list)
    for idx in tmp.index :
        tmpDict[tmp.at[idx,'단어명']] = [tmp.at[idx,'영문명'],tmp.at[idx,'약어명'],tmp.at[idx,'정의']]
    
    return tmpDict

In [50]:
# origin<-addition // duplicate => use origin
def merge_defaultdicts(origin,addition):
    for k,v in addition.items():
        if not k in origin:
            origin[k] = addition[k]
    return origin

In [5]:
def sumStrList(list_) :
    res = ''
    for txt in list_ :
        res += txt
    return res

In [7]:
# select_spacing_model : konlpy, spacing(DL model)
def select_spacing(input_, type_='konlpy') :
    if type_.lower() == 'konlpy' :
        return getNouns(input_)
    elif type_.lower() == 'spacing' :
        return predict(input_)
    else :
        raise print('we have only knolpy, spacing models')
        
# class spacing_konlpy()
def getNouns(input_) :
    from konlpy.tag import Okt
    tl = Okt().pos(input_)
    rl = []
    tmp =''

    for idx in range(len(tl)) :    
        if tl[idx][1] == 'Noun' :
            if tmp != '' :
                rl.append(tmp)
            rl.append(tl[idx][0])
            tmp = ''
        else :
            tmp += str(tl[idx][0])
    
    if tmp != '' :
        rl.append(tmp)
            
    return rl

In [8]:
def getSpacedKor(input_) :
    return '_'.join(input_)

In [9]:
def abbreviate(eng) :
    import re
    # except first_charactor
    tmp = eng.lower()[1:]
    
    # del vowel
    regex = re.compile('[^aeiou]')
    tmp = ''.join(regex.findall(eng))
    
    # del dupplicate_char
    for ch in tmp:
        dup = ch+ch
        tmp = re.sub(dup, ch, tmp)
    
    # del blank
    return re.sub(' ','',tmp).upper()
     

In [10]:
def isInDict(word,dict_) :
    isIn = False
    for l in dict_.values() :
        if word.upper() == l[1] :
            isIn = True
            break
            
    return isIn

In [11]:
def convert_abbr(eng,kor,dict_) :
    # abbreviate, isInDict
    
    tmp_abr = abbreviate(eng)
    isFound = False
    
    for idx in range(3,len(tmp_abr)+1) :
        if not isInDict(tmp_abr[:idx],dict_) :
            dict_[kor] = [eng,tmp_abr[:idx].upper(),get_definition(kor)]
            isFound = True
            return tmp_abr[:idx].upper(), dict_
            
    idx = 3
    tmp = eng.upper()
    while not isFound :
        if idx != len(eng) :
            for idx in range(3,len(eng)+1) :
                if not isInDict(eng[:idx],dict_) :
                    dict_[kor] = [eng,eng[:idx].upper(),get_definition(kor)]
                    isFound = True
                    return eng[:idx].upper(), dict_
        else :
            tmp += '_'
            if not isInDict(tmp,dict_) :
                dict_[kor], [eng,tmp.upper(),get_definition(kor)]
                isFount = True
                return tmp, dict_

In [12]:
def convert_kor2abr(input_, dict_ ) :
    tmp_spacedAbr = ''
    tmp_spacedEng = ''
    
    if input_ != [] :
        for idx in range(len(input_),0,-1) :
            tmpTxt = sumStrList(input_[:idx])
            if tmpTxt in dict_.keys() :
                tmp_spacedEng += dict_[tmpTxt][0] + '_'
                tmp_spacedAbr += dict_[tmpTxt][1] + '_'
                if idx != len(input_) :
                    sub_abr, sub_eng, dict_ = convert_kor2abr(input_[idx:], dict_)
                    tmp_spacedAbr += sub_abr
                    tmp_spacedEng += sub_eng
                break
                    
            if idx == 1 :
                tmp_eng = translate_by_papago_api(tmpTxt)
                # 약어변경 함수(convert_abbr)
                tmp_abr, dict_ = convert_abbr(tmp_eng,tmpTxt,dict_)
                tmp_spacedAbr += tmp_abr + '_'
                tmp_spacedEng += tmp_eng.upper() + '_'
                                
                if idx != len(input_) : 
                    sub_abr, sub_eng, dict_ = convert_kor2abr(input_[idx:],dict_)
                    tmp_spacedAbr += sub_abr
                    tmp_spacedEng += sub_eng
                
                    
        if tmp_spacedAbr[-1] == '_' :
            tmp_spacedAbr = tmp_spacedAbr[:-1]
            tmp_spacedEng = tmp_spacedEng[:-1]
            
    return tmp_spacedAbr, tmp_spacedEng, dict_
        

In [17]:
abr, eng, dict_ = convert_kor2abr(select_spacing('테스트놀이챔피언'),dict_)
abr, eng, 

('TEST_PLY_CHM', 'TEST_PLAY_CHAMPION')

In [18]:
dict_['챔피언']

['Champion',
 'CHM',
 '1. 운동 운동 종목 따위에서, 선수권을 보유하고 있는 사람. ‘선수권자’, ‘으뜸 선수’로 순화. ≒우승자.\n헤비급 세계 챔피언\n2. 기술 따위에서, 실력이 가장 뛰어난 사람.\n수학 경시대회 챔피언.']

In [63]:
def dict2sheet(path,input_,dict_) :
    # dict_col 단어명,영문명,약어명,정의
    import pandas as pd
    import numpy as np
    
    tmp = pd.DataFrame([])
    wrd = np.array([])
    eng = np.array([])
    abr = np.array([])
    def_ = np.array([])
    
    for k in dict_.keys() :
        wrd = np.append(wrd, k)
        eng = np.append(eng, dict_[k][0])
        abr = np.append(abr, dict_[k][1])
        def_ = np.append(def_, dict_[k][2])
        
    
    tmp['단어명'] = wrd
    tmp['영문명'] = eng
    tmp['약어명'] = abr
    tmp['정의'] = def_
    
    with pd.ExcelWriter(path, engine='xlsxwriter') as ew :
        input_.to_excel(ew, sheet_name='용어',index=False, )
        tmp.to_excel(ew, sheet_name='단어',index=False )

In [64]:
def convert_File(input_path, output_path,isUseDict = True) :
    import pandas as pd
    from collections import defaultdict
    
    
    
    input_ = pd.read_excel(input_path, sheet_name='용어')
    if output_path[input_path.rfind('/'):].find('.') == -1 :
        output_path += input_path[input_path.rfind('/'):]
    # np.NaN is float type
    input_ = input_.fillna("")
    
    # memory on dictionary
    dict_ = getWordDict(input_path)
    
    if isUseDict :
        defDict = getDefDict('C:/dev/행정표준용어.xls')
        dict_ = merge_defaultdicts(dict_, defDict)
        
    for idx in input_['약어명'].isnull().index :
        spaced_kor = select_spacing(input_.at[idx,'용어명'])
        abr, eng, dict_ = convert_kor2abr(spaced_kor,dict_)
        input_.at[idx,'약어명'] = abr
        input_.at[idx,'한글단어별'] = getSpacedKor(spaced_kor)
        input_.at[idx,'영문단어별'] = eng
        
    # input, dict_to_sheet
    dict2sheet(output_path,input_, dict_)
    

In [65]:
import pandas as pd

input_path = 'input/input.xlsx'
output_path = 'output/input.xlsx'
convert_File(input_path, output_path)

In [68]:
output_path[:output_path.rfind('/')]

'output'

In [49]:
d = getWordDict(input_path)
dd = getDefDict('C:/dev/행정표준용어.xls')
d = merge_defaultdicts(d, dd)
d

defaultdict(list,
            {'1': ['ONE', 'ONE', '숫자 1\r\n\r\n'],
             '10월': ['OCTOBER', 'OCT', '10월\r\n\r\n'],
             '119안전신고센터': ['NATIONAL 119 SECURITY CENTER',
              'N119SC',
              '범국민적으로 재해 및 안전에 대한 관심을 유도하고, 신고 체계를 확립할 목적으로 행정자치부에서 개설한 인터넷 안전관리시스템.\n'],
             '11월': ['NEVEMBER', 'NOV', '11월\r\n\r\n'],
             '12월': ['DECEMBER', 'DEC', '12월\r\n\r\n'],
             '13월': ['NEXT JANUARY',
              'NJAN',
              '차년도 1월 : 회계업무의 출납폐쇄월을 의미함\r\n\r\n'],
             '14월': ['NEXT FEBRUARY',
              'NFEB',
              '차년도 2월 : 회계업무의 출납폐쇄월을 의미함\r\n\r\n'],
             '1월': ['JANUARY', 'JAN', '1월\r\n\r\n'],
             '1종시설물': ['FIRST-CLASS FACILITIES',
              'FSLS',
              '1種施設物\n [법률정의] (시설물의 안전관리에 관한 특별법) 도로・철도・항만・댐・교량・터널・건축물등 공중의 이용편의와 안전을 도모하기 위하여 특별히 관리할 필요가 있거나 구조상 유지관리에 있어 고도의 기술이 필요하다고 인정하여 대통령령이 정하는 시설물.\n'],
             '1차냉각재': ['PRIMARY REFRIGERANT',
              'PRMRFRG',
           

In [69]:
tmp = ['안', '드로', '이드', '']

In [57]:
input_.fillna("")

,용어명,약어명,한글단어별,영문단어별
0,지대공미사일코드,,,
1,안드로이드접근코드,,,
2,사람개조설계명,,,
3,사전준비멀티미디어문제,,,
4,머신러닝딥러닝문제해결번호,,,
5,해결전략코드,,,
6,테스트완료용문제해결,,,
7,노쇼,,,
8,안드로이드백신개발부서,,,
9,멀티미디어문화콘텐츠설비제공업준비,,,


,용어명,약어명,한글단어별,영문단어별
0,False,True,True,True
1,False,True,True,True
2,False,True,True,True
3,False,True,True,True
4,False,True,True,True
5,False,True,True,True
6,False,True,True,True
7,False,True,True,True
8,False,True,True,True
9,False,True,True,True


In [24]:
# translate_function
def translate_by_selenium_naver(text) :
    from selenium import webdriver
    import urllib.request
    import requests
    import time
    from bs4 import BeautifulSoup
    
    chromedriver_dir = 'C:/dev/driver/chromedriver.exe'
    driver = webdriver.Chrome(chromedriver_dir)
    
    driver.get("https://papago.naver.com/")
    driver.implicitly_wait(3)
    driver.find_element_by_id('txtSource').send_keys(text)
    
    time.sleep(1)
    output_ = driver.find_element_by_id('txtTarget').text
    
    driver.close()
    
    return output_

def translate_by_selenium(text) :
    from selenium import webdriver
    import urllib.request
    import requests
    import time
    from bs4 import BeautifulSoup
    
    chromedriver_dir = 'C:\dev\driver\chromedriver.exe'
    driver = webdriver.Chrome(chromedriver_dir)

    driver.get("https://translate.google.com/")

    driver.find_element_by_id('sugg-item-en').click()
    driver.find_element_by_id('sugg-item-ko').click()

    driver.find_element_by_id('source').send_keys(text)

    time.sleep(1)
    source = driver.page_source
    soupData = BeautifulSoup(source,'html.parser')
    
    driver.close()

    return soupData.find('span',class_='tlid-translation translation').text



In [26]:
# directory_path end check (if use '/')
def dirEnd_check(path) :
    if path[-1] != '/' :
        path += '/'
    return path
    

In [ ]:
def convertFile_v2(inputDict, input_path,output_path='output') :
    import pandas as pd
    
    input_ = pd.read_excel(input_path)
    if output_path[input_path.rfind('/'):].find('.') == -1 :
        output_path += input_path[input_path.rfind('/'):]

    defDict = getDataDict()
    newDict = {}
    
    for idx in input_['output'].isnull().index :
        tmp, inputDict,defDict,newDict = convert_v4(input_.at[idx,'input'],inputDict,defDict,newDict)
        input_.output[idx] = tmp
        
    input_.to_excel(output_path,index=False)
    outputDict(newDict,output_path[:output_path.rfind('/')]+'/newDict.xlsx')    